task name: Resolve CustomMappingWithInMemoryCustomType
task description: Implement the provided C# code as a dotnet interactive notebook

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext();

In [ ]:
private class AlienBody
{
    public int Age { get; set; }
    public float Height { get; set; }
    public float Weight { get; set; }
    public int HandCount { get; set; }

    public AlienBody(int age, float height, float weight, int handCount)
    {
        Age = age;
        Height = height;
        Weight = weight;
        HandCount = handCount;
    }
}

private sealed class AlienTypeAttributeAttribute : DataViewTypeAttribute
{
    public int RaceId { get; }

    public AlienTypeAttributeAttribute(int raceId)
    {
        RaceId = raceId;
    }

    public override void Register()
    {
        DataViewTypeManager.Register(new DataViewAlienBodyType(RaceId),
            typeof(AlienBody), this);
    }

    public override bool Equals(DataViewTypeAttribute other)
    {
        if (other is AlienTypeAttributeAttribute alienTypeAttributeAttribute)
            return RaceId == alienTypeAttributeAttribute.RaceId;
        return false;
    }

    public override int GetHashCode() => RaceId.GetHashCode();
}

private class AlienHero
{
    public string Name { get; set; }

    [AlienTypeAttribute(100)]
    public AlienBody One { get; set; }

    [AlienTypeAttribute(200)]
    public AlienBody Two { get; set; }

    public AlienHero()
    {
        Name = "Unknown";
        One = new AlienBody(0, 0, 0, 0);
        Two = new AlienBody(0, 0, 0, 0);
    }

    public AlienHero(string name,
        int age, float height, float weight, int handCount,
        int anotherAge, float anotherHeight, float anotherWeight, int
            anotherHandCount)
    {
        Name = name;
        One = new AlienBody(age, height, weight, handCount);
        Two = new AlienBody(anotherAge, anotherHeight, anotherWeight,
            anotherHandCount);
    }
}

private class DataViewAlienBodyType : StructuredDataViewType
{
    public int RaceId { get; }

    public DataViewAlienBodyType(int id) : base(typeof(AlienBody))
    {
        RaceId = id;
    }

    public override bool Equals(DataViewType other)
    {
        if (other is DataViewAlienBodyType otherAlien)
            return otherAlien.RaceId == RaceId;
        return false;
    }

    public override int GetHashCode()
    {
        return RaceId.GetHashCode();
    }
}

private class SuperAlienHero
{
    public string Name { get; set; }

    [AlienTypeAttribute(007)]
    public AlienBody Merged { get; set; }

    public SuperAlienHero()
    {
        Name = "Unknown";
        Merged = new AlienBody(0, 0, 0, 0);
    }
}

private class AlienFusionProcess
{
    public static void MergeBody(AlienHero input, SuperAlienHero output)
    {
        output.Name = "Super " + input.Name;
        output.Merged.Age = input.One.Age + input.Two.Age;
        output.Merged.Height = input.One.Height + input.Two.Height;
        output.Merged.Weight = input.One.Weight + input.Two.Weight;
        output.Merged.HandCount = input.One.HandCount + input.Two.HandCount;
    }

    public static Action<AlienHero, SuperAlienHero> GetMapping()
    {
        return MergeBody;
    }
}

In [ ]:
var tribe = new List<AlienHero>() { new AlienHero("ML.NET", 2, 1000, 2000, 3000, 4000, 5000, 6000, 7000) };

In [ ]:
var tribeDataView = mlContext.Data.LoadFromEnumerable(tribe);
var pipeline = mlContext.Transforms.CustomMapping(AlienFusionProcess
            .GetMapping(), contractName: null);

var model = pipeline.Fit(tribeDataView);
var tribeTransformed = model.Transform(tribeDataView);

In [ ]:
var firstAlien = mlContext.Data.CreateEnumerable<SuperAlienHero>(tribeTransformed, false).First();

Console.WriteLine("We got a super alien with name " + firstAlien.Name + ", age " + firstAlien.Merged.Age + ", " + "height " + firstAlien.Merged.Height + ", weight  " + firstAlien.Merged.Weight + ", and " + firstAlien.Merged.HandCount + " hands.");

In [ ]:
var engine = mlContext.Model.CreatePredictionEngine<AlienHero, SuperAlienHero>(model);

var alien = new AlienHero("TEN.LM", 1, 2, 3, 4, 5, 6, 7, 8);
var superAlien = engine.Predict(alien);
Console.Write("We got a super alien with name " + superAlien.Name + ", age " + superAlien.Merged.Age + ", height " + superAlien.Merged.Height + ", weight " + superAlien.Merged.Weight + ", and " + superAlien.Merged.HandCount + " hands.");